In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
cat1 = 'Sculptors'
cat2 = 'Computer_Scientists'

In [3]:
def get_people(profession):
    sparql = SPARQLWrapper("https://dbpedia.org/sparql")
    
    modified_profession = profession.replace(' ', '_').replace("'", '%27')
    
    query = """
            Select Distinct ?person Where {
                ?person dbo:occupation dbr:%s .
            }
            """ % modified_profession
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    ret = sparql.queryAndConvert()
    return(ret['results']['bindings'])


In [20]:
def get_rdf_triplets(person):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    encode_name = quote(person.replace('', '_'))
    dbpedia_resource = "http://dbpedia.org/resource/" + encode_name
    query = "SELECT ?predicate ?object WHERE { <" + dbpedia_resource + "> ?predicate ?object.}"
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    triples = []
    for result in results["results"]["bindings"]:
        subj = "<" + dbpedia_resource + ">"
        pred = result["predicate"]["value"]
        obj = result["object"]["value"]
        triples.append({'subject': subj, 'predicate': pred, 'object': obj})
    return triples

In [19]:
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import os

def extract_category_facts(category_name, output_file):
    # Initialize SPARQL endpoint
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    dbpedia_resource = "http://dbpedia.org/resource/" + category_name.split(':')[1]
    
    # if not os.path.exists(output_file):
    #     os.makedirs(output_file)
    
    # Construct the SPARQL query to retrieve RDF triples related to the category
    # query = """
    # PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    # PREFIX dcterms: <http://purl.org/dc/terms/>
    # SELECT ?subject ?predicate ?object
    # WHERE {
    #     ?subject dcterms:subject <http://dbpedia.org/resource/Category:%s> .
    #     ?subject ?predicate ?object .
    #     FILTER regex(str(?subject), "http://dbpedia.org/resource/") .
    # }
    # LIMIT 100
    # """ % category_name.split(':')[1]

    query = "SELECT ?predicate ?object WHERE { <" + dbpedia_resource + "> ?predicate ?object.}"
    
    # Set the SPARQL query and return format
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    # Execute the SPARQL query
    results = sparql.query().convert()
    
    # Extract and store RDF triples in a list
    triples = []
    for result in results["results"]["bindings"]:
        # subject = result["subject"]["value"]
        subj = "<"+dbpedia_resource+">"
        predicate = result["predicate"]["value"]
        obj = result["object"]["value"]
        triples.append({"subject": subj, "predicate": predicate, "object": obj})
    
    # Save RDF triples to a JSON file
    with open(output_file, 'w') as f:
        json.dump(triples, f, indent=4)

# Example usage:
category_name = "Category:Computer_Scientists"
output_file = "computers.json"
extract_category_facts(category_name, output_file)


In [16]:
import json
import os
from SPARQLWrapper import SPARQLWrapper, JSON

def get_people(profession):
    sparql = SPARQLWrapper("https://dbpedia.org/sparql")
    
    modified_profession = profession.replace(' ', '_').replace("'", '%27')
    
    query = """
            SELECT DISTINCT ?person WHERE {
                ?person dbo:occupation dbr:%s .
            }
            """ % modified_profession
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    ret = sparql.query().convert()
    return [result['person']['value'].split('/')[-1] for result in ret['results']['bindings']]

def get_rdf_triples(person):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    dbpedia_resource = "http://dbpedia.org/resource/" + person
    query = """
            SELECT ?predicate ?object WHERE {
                <""" + dbpedia_resource + """> ?predicate ?object.
            }
            """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    triples = []
    for result in results["results"]["bindings"]:
        subj = "<" + dbpedia_resource + ">"
        pred = result["predicate"]["value"]
        obj = result["object"]["value"]
        triples.append({'subject': subj, 'predicate': pred, 'object': obj})
    return triples

def save_rdf_triples_by_profession(profession, output_dir):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Get people by profession
    people = get_people(profession)
    
    # Get RDF triples for each person and save to a separate file
    for person in people:
        triples = get_rdf_triples(person)
        output_file = os.path.join(output_dir, f"{person}.json")
        with open(output_file, 'w') as f:
            json.dump(triples, f, indent=4)

# Example usage:
profession = "Computer_Scientists"
output_dir = "computer_rdf"
save_rdf_triples_by_profession(profession, output_dir)


In [15]:
save_rdf_triples_by_profession('Computer_Scientists', 'computer_scientists_rdf')